<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mistral 7B

Mistral 7B is a new state-of-the-art open-source model. Here are some interesting facts about it

* One of the strongest open-source models, of all sizes
* Strongest model in the 1-20B parameter range models
* Does decently in code-related tasks
* Uses Windowed attention, allowing to push to 200k tokens of context if using Rope (needs 4 A10G GPUs for this)
* Apache 2.0 license

As for the integrations status:
* Integrated into `transformers`
* You can use it with a server or locally (it's a small model after all!)
* Integrated into popular tools tuch as TGI and VLLM


Two models are released: a [base model](https://huggingface.co/mistralai/Mistral-7B-v0.1) and a [instruct fine-tuned version](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1). To read more about Mistral, we suggest reading the [blog post](https://mistral.ai/news/announcing-mistral-7b/).

In this Colab, we'll experiment with the Mistral model using an API. There are three ways we can use it:

* **Free API:** Hugging Face provides a free Inference API for all its users to try out models. This API is rate limited but is great for quick experiments.
* **PRO and Enterprise API:** Hugging Face provides an open API for all its PRO users.  Subscribing to the Pro Inference API costs $9/month and allows you to experiment with many large models, such as Llama 2 and SDXL. Read more about it [here](https://huggingface.co/blog/inference-pro).
* **Inference Endpoints:** For enterprise and production-ready cases. You can deploy it with 1 click [here](https://ui.endpoints.huggingface.co/catalog).

This demo does not require GPU Colab, just CPU. You can grab your token at https://huggingface.co/settings/tokens.

**This colab shows how to use HTTP requests as well as building your own chat demo for Mistral.**

## Doing curl requests


In this notebook, we'll experiment with the instruct model, as it is trained for instructions. As per [the model card](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1), the expected format for a prompt is as follows

From the model card

> In order to leverage instruction fine-tuning, your prompt should be surrounded by [INST] and [\INST] tokens. The very first instruction should begin with a begin of sentence id. The next instructions should not. The assistant generation will be ended by the end-of-sentence token id.

```
<s>[INST] {{ user_msg_1 }} [/INST] {{ model_answer_1 }}</s> [INST] {{ user_msg_2 }} [/INST] {{ model_answer_2 }}</s>
```

Note that models can be quite reactive to different prompt structure than the one used for training, so watch out for spaces and other things!

We'll start an initial query without prompt formatting, which works ok for simple queries.

In [ ]:
!curl https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1 \
  --header "Content-Type: application/json" \
	-X POST \
	-d '{"inputs": "Explain ML as a pirate", "parameters": {"max_new_tokens": 50}}' \
	-H "Authorization: Bearer API_TOKEN"

{"error":"Authorization header is correct, but the token seems invalid"}

## Programmatic usage with Python

You can do simple `requests`, but the `huggingface_hub` library provides nice utilities to easily use the model. Among the things we can use are:

* `InferenceClient` and `AsyncInferenceClient` to perform inference either in a sync or async way.
* Token streaming: Only load the tokens that are needed
* Easily configure generation params, such as `temperature`, nucleus sampling (`top-p`), repetition penalty, stop sequences, and more.
* Obtain details of the generation (such as the probability of each token or whether a token is the last token).

In [ ]:
%%capture
!pip install huggingface_hub gradio

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(
    "mistralai/Mistral-7B-Instruct-v0.1"
)

prompt = """<s>[INST] What is your favourite condiment?  [/INST]</s>
"""

res = client.text_generation(prompt, max_new_tokens=95)
print(res)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



I don't have personal experiences or preferences. However, I can tell you that people's favorite condiments can vary widely based on personal taste and cultural background. Some popular condiments include ketchup, mustard, mayonnaise, hot sauce, soy sauce, and olive oil.


We can also use [token streaming](https://huggingface.co/docs/text-generation-inference/conceptual/streaming). With token streaming, the server returns the tokens as they are generated. Just add `stream=True`.

In [ ]:
res = client.text_generation(prompt, max_new_tokens=35, stream=True, details=True, return_full_text=False)
for r in res: # this is a generator
  # print the token for example
  print(r)
  continue

TextGenerationStreamResponse(token=Token(id=13, text='\n', logprob=-0.44873047, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=28737, text='I', logprob=-1.5058594, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=949, text=' don', logprob=-1.3095703, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=28742, text="'", logprob=-0.01713562, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=28707, text='t', logprob=-1.2755394e-05, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=506, text=' have', logprob=-0.54345703, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=3327, text=' personal', logprob=-0.33081055, special=False), generated_text=None, details=None)
TextGenerationStreamResponse(token=Token(id=9021, text=' exp

Let's now try a multi-prompt structure

In [ ]:
def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

In [ ]:
message = "And what do you think about it?"
history = [["What is your favourite condiment?", "My favorite condiment is ketchup. It's versatile, tasty, and goes well with a variety of foods."]]

format_prompt(message, history)

"<s>[INST] What is your favourite condiment? [/INST] My favorite condiment is ketchup. It's versatile, tasty, and goes well with a variety of foods.</s> [INST] And what do you think about it? [/INST]"

## End-to-end demo

Let's now build a Gradio demo that takes care of:

* Handling multiple turns of conversation
* Format the prompt in correct structure
* Allow user to specify/modify the parameters
* Stop the generation

Just run the following cell and have fun!

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def generate(
    prompt, history, temperature=0.9, max_new_tokens=256, top_p=0.95, repetition_penalty=1.0,
):
    temperature = float(temperature)
    if temperature < 1e-2:
        temperature = 1e-2
    top_p = float(top_p)

    generate_kwargs = dict(
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        seed=42,
    )

    formatted_prompt = format_prompt(prompt, history)

    stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=True, details=True, return_full_text=False)
    output = ""

    for response in stream:
        output += response.token.text
        yield output
    return output


additional_inputs=[
    gr.Slider(
        label="Temperature",
        value=0.9,
        minimum=0.0,
        maximum=1.0,
        step=0.05,
        interactive=True,
        info="Higher values produce more diverse outputs",
    ),
    gr.Slider(
        label="Max new tokens",
        value=256,
        minimum=0,
        maximum=8192,
        step=64,
        interactive=True,
        info="The maximum numbers of new tokens",
    ),
    gr.Slider(
        label="Top-p (nucleus sampling)",
        value=0.90,
        minimum=0.0,
        maximum=1,
        step=0.05,
        interactive=True,
        info="Higher values sample more low-probability tokens",
    ),
    gr.Slider(
        label="Repetition penalty",
        value=1.2,
        minimum=1.0,
        maximum=2.0,
        step=0.05,
        interactive=True,
        info="Penalize repeated tokens",
    )
]

with gr.Blocks() as demo:
    gr.ChatInterface(
        generate,
        additional_inputs=additional_inputs,
    )

demo.queue().launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6babc0e4bd0841d57a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6babc0e4bd0841d57a.gradio.live


## What's next?

* Try out Mistral 7B in this [free online Space](https://huggingface.co/spaces/osanseviero/mistral-super-fast)
* Deploy Mistral 7B Instruct with one click [here](https://ui.endpoints.huggingface.co/catalog)
* Deploy in your own hardware using https://github.com/huggingface/text-generation-inference
* Run the model locally using `transformers`

In [ ]:
! pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.6 MB/s eta 0:00:00


In [ ]:
from docx import Document
from huggingface_hub import InferenceClient

client = InferenceClient("mistralai/Mistral-7B-Instruct-v0.1")

# Sample prompt
prompt = """<s>[INST] What is your favourite condiment?  [/INST]</s>
"""

prompt = [
    "Crafting the Introduction:\n- Develop a captivating opening statement that highlights the significance of addressing mental health in Iran.\n- Mention the success of Safecast Japan and draw parallels to the potential positive impact on societal well-being in Iran.",

    "Explain the Purpose:\n- Clearly articulate the objective of the psychological phase, emphasizing the importance of understanding and addressing mental health challenges in the country.",

    "Provide Context:\n- Briefly explain the societal context in Iran, touching upon relevant challenges and the potential benefits of the project.",

    "Utilize Visuals:\n- Prepare compelling visuals, such as infographics or video clips, showcasing the successes of Safecast Japan and the potential positive outcomes for Iran.",


]
# Generate response using Mistral
res = client.text_generation(prompt, max_new_tokens=95)
print(res)

# Save response in DOCX file with a suggested related topic as the subtitle
doc = Document()
doc.add_heading('Document Title', 0)  # Title placeholder

# Sample prompt and response
prompt_response_pairs = [
    ("What is your favourite condiment?", "My favorite condiment is ketchup. It's versatile, tasty, and goes well with a variety of foods."),
    # Add more prompt-response pairs as needed
]

for i, (user_prompt, bot_response) in enumerate(prompt_response_pairs):
    # Generate formatted prompt for Mistral
    formatted_prompt = format_prompt(bot_response, [(user_prompt, bot_response)])

    # Generate Mistral response
    response = client.text_generation(formatted_prompt, max_new_tokens=256, stream=True, details=True, return_full_text=False)

    # Save response in DOCX file
    doc.add_heading(f'Related Topic {i + 1}', level=1).bold = True
    doc.add_heading(user_prompt, level=2).bold = True
    doc.add_paragraph(bot_response)

# Save the DOCX file
doc.save("safecast_iran_project.docx")

# Prompt to suggest the title for the DOCX file
title_prompt = "Suggest a Title for the Document:"
title_response = client.text_generation(title_prompt, max_new_tokens=50)
print(title_response)

#made by lmsys :

In [ ]:
!pip install huggingface-hub python-docx gradio

In [ ]:
curriculum_prompts = [
    f"Curriculum Title: Civil Conflict Resolution and Transformative Scenario Planning for the Iranian Opposition",
    f"Module 1: Understanding Conflict Dynamics (Weeks 1-2)\n"
    f"1.1 Introduction to Conflict Resolution\n"
    f"1.2 Understanding the Iranian Political Landscape\n"
    f"Conflict Resolution Approaches",
    f"Module 2: Communication and Dialogue (Weeks 3-4)\n"
    f"2.1 Effective Communication Skills\n"
    f"2.2 Constructive Dialogue Techniques\n"
    f"2.3 Cultural Sensitivity in Communication",
    f"Module 3: Mediation and Negotiation (Weeks 5-6)\n"
    f"3.1 Mediation Process\n"
    f"3.2 Negotiation Strategies\n"
    f"3.3 Case Studies in Mediation and Negotiation",
    f"Module 4: Advocacy and Civil Engagement (Weeks 7-8)\n"
    f"4.1 Advocacy for Conflict Resolution\n"
    f"4.2 Leveraging International Support\n"
    f"4.3 Community Engagement",
    f"Module 5: Ethical Considerations (Weeks 9-10)\n"
    f"5.1 Ethics in Conflict Resolution\n"
    f"5.2 Navigating Legal Frameworks",
    f"Module 6: Building Resilience and Sustainable Peace (Weeks 11-12)\n"
    f"6.1 Psychosocial Support\n"
    f"6.2 Sustainable Peacebuilding",
    f"Module 7: Transformative Scenario Planning (Weeks 13-14)\n"
    f"7.1 Introduction to Transformative Scenario Planning\n"
    f"7.2 Understanding the Iranian Political Landscape for Scenario Planning\n"
    f"7.3 Analysis of Current Conflict Dynamics for Scenario Planning",
    f"Module 8: Tools and Techniques of Transformative Scenario Planning (Weeks 15-16)\n"
    f"8.1 Scenario Building Techniques\n"
    f"8.2 Systems Thinking and Complexity in Conflict\n"
    f"8.3 Data Collection and Analysis for Scenario Planning",
    f"Module 9: Visioning and Goal Setting for Scenario Planning (Weeks 17-18)\n"
    f"9.1 Envisioning a Desired Future\n"
    f"9.2 Setting Transformative Goals for Scenario Planning",
    f"Module 10: Stakeholder Engagement and Collaboration for Scenario Planning (Weeks 19-20)\n"
    f"10.1 Identifying and Mapping Stakeholders for Scenario Planning\n"
    f"10.2 Facilitating Collaborative Processes for Scenario Planning",
    f"Module 11: Scenario Testing and Rehearsal (Weeks 21-22)\n"
    f"11.1 Testing Scenarios Through Simulation\n"
    f"11.2 Adaptive Strategy Development for Scenario Planning",
    f"Module 12: Communication and Advocacy for Scenario Planning (Weeks 23-24)\n"
    f"12.1 Communicating Transformative Visions for Scenario Planning\n"
    f"12.2 Advocacy for Conflict Transformation in Scenario Planning",
    f"Module 13: Implementation and Monitoring (Weeks 25-26)\n"
    f"13.1 Action Planning and Implementation for Scenario Planning\n"
    f"13.2 Monitoring and Adaptive Management for Scenario Planning",
    f"Module 14: Reflection and Future Directions (Weeks 27-28)\n"
    f"14.1 Course Reflection and Evaluation\n"
    f"14.2 Developing a Sustainable Path Forward",
    f"Assessment:\n"
    f"Active participation in discussions and activities\n"
    f"Case study analyses\n"
    f"Mediation and negotiation simulations\n"
    f"Advocacy and community engagement project\n"
    f"Scenario-building exercises\n"
    f"Stakeholder engagement and collaboration project\n"
    f"Action plan development and presentation\n"
    f"Final reflection and vision for the future"
]

In [ ]:

from huggingface_hub import InferenceClient
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

# Initialize the InferenceClient with your chosen model
client = InferenceClient("mistralai/Mistral-7B-Instruct-v0.1")

# Define a prompt for generating the document title
title_prompt = "<s>[INST] Suggest a title for a document covering a curriculum on Civil Conflict Resolution and Transformative Scenario Planning for the Iranian Opposition. [/INST]</s>"

# Generate the title
try:
    title_res = client.text_generation(title_prompt, max_new_tokens=35)
    if title_res and 'generated_text' in title_res[0]:
        doc_title = title_res[0]['generated_text'].strip()
    else:
        raise ValueError("No generated text found in the response.")
except Exception as e:
    print(f"An error occurred while generating the title: {e}")
    doc_title = "Civil Conflict Resolution and Scenario Planning Curriculum"

# Create a new Word document
doc = Document()

def add_heading(doc, text, level=1):
    heading = doc.add_heading(text, level=level)
    heading.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    run = heading.runs[0]
    run.font.size = Pt(14 if level == 1 else 12)

def add_paragraph(doc, text):
    paragraph = doc.add_paragraph(text)
    run = paragraph.runs[0]
    run.font.size = Pt(11)

# Add the generated title to the document
add_heading(doc, doc_title, level=1)

# Example: Add content for a couple of modules
curriculum_prompts_0 = [
    "Module 1: Understanding Conflict Dynamics (Weeks 1-2)",
    # ... Add other modules here
]

# Generate content for each prompt and add it to the document
for index, module_prompt in enumerate(curriculum_prompts):
    prompt = f"<s>[INST] {module_prompt} [/INST]</s>"
    try:
        res = client.text_generation(prompt, max_new_tokens=95)
        if res and 'generated_text' in res[0]:
            add_heading(doc, f"Module {index+1}", level=2)
            add_paragraph(doc, res[0]['generated_text'])
        else:
            add_paragraph(doc, f"Content for {module_prompt} could not be generated.")
    except Exception as e:
        print(f"An error occurred while generating content for {module_prompt}: {e}")
        add_paragraph(doc, f"Content for {module_prompt} could not be generated.")

# Save the document with the generated title as the filename
doc.save(f"{doc_title}.docx")

An error occurred while generating the title: No generated text found in the response.
An error occurred while generating content for Module 10: Stakeholder Engagement and Collaboration for Scenario Planning (Weeks 19-20)
10.1 Identifying and Mapping Stakeholders for Scenario Planning
10.2 Facilitating Collaborative Processes for Scenario Planning: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1 (Request ID: 4Yjvo7tl_KxoGcejwcgeu)

Rate limit reached. Please log in or use your apiToken
An error occurred while generating content for Module 11: Scenario Testing and Rehearsal (Weeks 21-22)
11.1 Testing Scenarios Through Simulation
11.2 Adaptive Strategy Development for Scenario Planning: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1 (Request ID: 7R66E7LqJotnth14phocz)

Rate limit reached. Please log in or use your apiToken
An error occur

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

In [ ]:
free_in_GB = int(torch.cuda.mem_get_info()[0] / 1024**3)
max_memory = f"{free_in_GB-2}GB"

n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}
max_memory

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b",
    load_in_8bit=True,
    device_map={"":0},
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    "tiiuae/falcon-7b",
)

In [ ]:
! pip install python-docx

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Path to the local directory where the model is saved
local_model_dir = "./mistralai/Mistral-7B-Instruct-v0.1"
local_model_dir = "tiiuae/falcon-40b"
# Load tokenizer and model from local directory
tokenizer = AutoTokenizer.from_pretrained(local_model_dir)
model = AutoModelForCausalLM.from_pretrained(local_model_dir)

# Function to generate text using the loaded model
def generate_text(prompt, max_length=50):
    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    # Generate a sequence of text after the prompt
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

    # Decode the output sequences to text
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    return generated_text

# Example usage
prompt = "Hello, my name is"
generated_text = generate_text(prompt)
print(generated_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

pytorch_model-00001-of-00009.bin:   0%|          | 0.00/9.50G [00:00<?, ?B/s]

pytorch_model-00002-of-00009.bin:   0%|          | 0.00/9.51G [00:00<?, ?B/s]